For each character calculate:
- Polymorphism: average load, average number of languages per state, number of languages with a polymorphism
- Number of states: num states, num informative states
- Homoplasy: num homoplastic states, num informative homoplastic states, number of languages exhibiting a homoplastic state [note: can’t do for IE]

In [9]:
E_FACTOR = [0.8]
H_FACTOR = ["0.1"]
C_FACTOR = [0.25, 0.5, 1,3, ]
POLYMORPHISM = ["no", "low", "high"]

TREES = 16
REPLICAS = 4

OUTPUT_FOLDER = 'simulated_data_theorypaper'

N_HP_STATES = 1
HP_STATES = [
    str(i) for i in range(N_HP_STATES)
]

In [10]:
import numpy as np
from collections import Counter

In [11]:
def get_statistics(row):
    # row is a dict of language -> list of states

    # polymorphism statistics 

    # average load defined as the avg. number of states per language
    avg_load = np.array([ len(v) for _, v in row.items() ]).mean()
    # ratio of langs with a polymorphism
    langs_with_poly = len([ k for k, v in row.items() if len(v) > 1 ])
    ratio_langs_with_poly = langs_with_poly / len(row)
    # avg. no. of languages per state
    state_lang_count = Counter() # counter of state -> how many languages have that state
    for k, v in row.items():
        for s in v:
            state_lang_count[s] += 1
    avg_langs_per_state = np.array(list(state_lang_count.values())).mean()

    # State statistics
    num_states = len(state_lang_count)
    num_informative_states = len([ k for k, v in state_lang_count.items() if v > 1 ]) # number of states with more than one character

    # num homoplastic states 
    hp_state_count = Counter({
        k: state_lang_count[k] for k in HP_STATES
        if k in state_lang_count 
    })
    n_hp_states = len(hp_state_count)
    # average size of homoplastic states
    avg_hp_size = sum(hp_state_count.values()) / len(hp_state_count) if len(hp_state_count) > 0 else None


    return (
        avg_load, 
        ratio_langs_with_poly, 
        avg_langs_per_state,
        num_states, 
        num_informative_states,
        n_hp_states,
        avg_hp_size
    )
            




Quartet statistics: 
- EVANS-ONE-K: get number of distinct quartets / n^4
- EVANS-ALL-K: get number of ties 

In [12]:
import sys
import os
import pandas as pd
sys.path.append(os.path.abspath('../lib'))
from getQuartets import get_new_omp
import scipy.special

def get_quartet_statistics(
    csv_path: str
):
    # all information is calculated with NO information of the HP state
    N = len(pd.read_csv(csv_path).drop(columns= ['id', 'feature', 'weight']).columns)
    (md, evans_one_res) = get_new_omp(
        csv_path = csv_path,
        mode = 12,
    ) 
    evans_all_res, ties, unique_best = md['votes'], md['ties'], md['unique_best']
    evans_all_covered_tuples = set([
        tuple(sorted(t))
        for t in evans_all_res.keys()
    ])

    n_c_4 = scipy.special.comb(N, 4, exact=True)
    ratio_distinct_evans_one = len(evans_one_res) / n_c_4
    ratio_distinct_evans_all = len(evans_all_res) / n_c_4
    ratio_covered_tuples_evans_all = len(evans_all_covered_tuples) / n_c_4
    assert (2 * ties + unique_best) == len(evans_one_res)
    ratio_evans_one_ties = ties / (len(evans_one_res))
    return ratio_distinct_evans_one, ratio_distinct_evans_all, ratio_evans_one_ties, ratio_covered_tuples_evans_all

In [16]:
from pathlib import Path
import os
import pandas as pd 
OMP = Path(os.getenv("TALLIS")) / "OneMostProb"

def get_single_dataset_stats(
    csv_path: str,
):
    dataset = pd.read_csv(csv_path).drop(columns= ['id', 'feature', 'weight'])

    dataset.head()

    row_stats = []

    for i, row in dataset.iterrows():
        row = row.to_dict()
        row = {
            k: str(v).split('/') for k, v in row.items()
        }
        row_stats.append(get_statistics(row))

    

    dataset_stats = pd.DataFrame.from_records(
        data = row_stats,
        columns = [
            "avg_load",
            "ratio_langs_with_poly",
            "avg_langs_per_state",
            "num_states",
            "num_informative_states",
            "num_hp_states",
            "avg_hp_size",
        ]
    ).mean(axis=0)
    ratio_distinct_evans_one, ratio_distinct_evans_all, ratio_evans_one_ties, ratio_covered_tuples_evans_all= get_quartet_statistics(csv_path)
    dataset_stats['ratio_distinct_evans_one'] = ratio_distinct_evans_one
    dataset_stats['ratio_distinct_evans_all'] = ratio_distinct_evans_all
    dataset_stats['ratio_evans_one_ties'] = ratio_evans_one_ties
    dataset_stats['ratio_covered_tuples_evans_all'] = ratio_covered_tuples_evans_all
    return dataset_stats.to_frame().T

def get_single_sim_dataset_stats(
    poly: str,
    hf: float,
    ef: int,
    cf: int,
    tree: int,
    rep: int,
):
    csv_path = OMP / "example" / OUTPUT_FOLDER / f"{poly}_{hf}_{ef}_{cf}" / f"sim_tree{tree}_{rep}.csv"
    return get_single_dataset_stats(csv_path)

    

In [17]:
from itertools import product
from tqdm import tqdm

In [ ]:
different_conditions_data = []
for (ef, hf, cf, poly) in tqdm(product(E_FACTOR, H_FACTOR, C_FACTOR, POLYMORPHISM)):
    for tree in range(1, 9):
        dataset_stats = get_single_sim_dataset_stats(
            poly = poly,
            ef = ef,
            hf = hf,
            cf = cf,
            tree = tree,
            rep = 1,
        )
        dataset_stats["poly"] = poly
        dataset_stats["ef"] = ef
        dataset_stats["hf"] = hf
        dataset_stats["cf"] = cf
        different_conditions_data.append(dataset_stats)
    #     if tree > 2:
    #         break
    # break
    # means = condition_data.mean(axis=0)

# print(different_conditions_data)
different_conditions_data = pd.concat(different_conditions_data, ignore_index=True)
# print(different_conditions_data.head())
   

6it [00:06,  1.14s/it]

In [ ]:
different_conditions_data.head()

,avg_load,ratio_langs_with_poly,avg_langs_per_state,num_states,num_informative_states,num_hp_states,avg_hp_size,ratio_distinct_evans_one,ratio_distinct_evans_all,ratio_evans_one_ties,ratio_covered_tuples_evans_all,poly,ef,hf,cf
0,1.040833,0.037917,3.000281,12.368750,4.800000,0.096875,3.451613,0.968801,1.005546,0.006704,0.962379,mod,0.8,0.1,1
1,1.038958,0.036146,3.350280,11.578125,4.518750,0.087500,5.214286,0.983470,1.040321,0.010797,0.972852,mod,0.8,0.1,1
2,1.036250,0.033542,2.794377,13.190625,4.909375,0.093750,3.500000,0.960226,1.012844,0.003724,0.956650,mod,0.8,0.1,1
3,1.041042,0.036771,3.013845,12.312500,4.315625,0.093750,3.800000,0.933881,0.972487,0.009417,0.925634,mod,0.8,0.1,1
4,1.042396,0.038229,3.314765,11.484375,3.503125,0.093750,5.400000,0.948586,1.029922,0.022003,0.928334,mod,0.8,0.1,1


RT-Screened-1 Stats

In [ ]:
rt_screened_1_stats = get_single_dataset_stats(OMP / "example" / "rt_2025_poly" / "rt_2025_poly_screened_lv_1.csv").mean(axis=0)

In [ ]:
rt_screened_1_stats['avg_load']

1.0798266461624493

In [ ]:
TARGETS = [
    "avg_load",
    "ratio_langs_with_poly",
    "avg_langs_per_state",
    "num_states",
    "num_informative_states",
    "ratio_distinct_evans_one",
    "ratio_distinct_evans_all",
    "ratio_evans_one_ties",
    "num_hp_states",
    "avg_hp_size"
]

In [ ]:
import seaborn as sns

In [ ]:
def plot_by_chr_and_hp(
    chrf,
    tgt, 
    tgt_name,
    draw_ie_ref = True,
):
    fg = sns.catplot(
        data = different_conditions_data[(different_conditions_data['cf'] == chrf)],
        col = 'ef',
        x = 'poly',
        y = tgt,
        hue = 'hf',
        kind = "point",
        aspect=0.5,
    )
    if draw_ie_ref:
        fg.refline(y = rt_screened_1_stats[tgt])
    fg.set_titles(
        col_template="Evo. Factor = {col_name}",
    )
    # fg.set_ylabels(tgt_name)
    fg.set_xlabels("Polymorphism")
    fg.set_xticklabels(rotation=60)
    fg.figure.subplots_adjust(top=0.8)
    fg.figure.suptitle(f"1 homoplastic state: {tgt_name} on {chrf * 320} characters", wrap=True)
    # fg.tight_layout()
    fg.figure.savefig(OMP / "figs" / f"stat-{tgt}.png")

In [ ]:
plot_by_chr_and_hp(1, "avg_load", 'Average Polymorphism Load')
plot_by_chr_and_hp(1, "ratio_langs_with_poly", 'Ratio of languages with a polymorphic state')
plot_by_chr_and_hp(1, "avg_langs_per_state", 'Average languages per state')
plot_by_chr_and_hp(1, "num_states", 'Average number of states')
plot_by_chr_and_hp(1, "num_informative_states", 'Average number of informative (big) states')
plot_by_chr_and_hp(1, "ratio_distinct_evans_one", 'Average number of distinct informative quartets in EVANS-ONE / nC4')
plot_by_chr_and_hp(1, "ratio_distinct_evans_all", 'Average number of distinct informative quartets in EVANS-ALL / nC4')
plot_by_chr_and_hp(1, "ratio_covered_tuples_evans_all", 'Ratio of four languages (nC4 in total) that have at least one quartet as generated by EVANS-ALL')
plot_by_chr_and_hp(1, "ratio_evans_one_ties", 'No. of times that two topologies appear for the same set of four taxa / number of quartets generated under EVANS-ONE')
plot_by_chr_and_hp(1, "num_hp_states", 'Avg. Number of homoplastic states that appear.', draw_ie_ref=False)
plot_by_chr_and_hp(1, "avg_hp_size", 'Average homoplastic state size', draw_ie_ref= False)

NameError: name 'plot_by_chr_and_hp' is not defined